In [1]:
import cv2
import numpy as np
import os
import pickle
from tqdm.notebook import tqdm
from pathlib import Path

# 1. generate video for goyal dataset

In [2]:
obs_info_dict_path = "train_data/training_observation_dict_raw.pkl"
test_pos_obs_info_dict_path = "test_data/testing_positive_observation_dict_raw.pkl"
test_neg_obs_info_dict_path = "test_data/testing_negative_observation_dict_raw.pkl"

In [3]:
with open(obs_info_dict_path, 'rb') as f:
    obs_info_dict = pickle.load(f)
with open(test_pos_obs_info_dict_path, 'rb') as f:
    obs_info_dict_p = pickle.load(f)
with open(test_neg_obs_info_dict_path, 'rb') as f:
    obs_info_dict_pp = pickle.load(f)

In [4]:
list(obs_info_dict.items())[0]

(0,
 ['1458/3900.jpeg',
  '1458/3901.jpeg',
  '1458/3902.jpeg',
  '1458/3903.jpeg',
  '1458/3904.jpeg',
  '1458/3905.jpeg',
  '1458/3906.jpeg',
  '1458/3907.jpeg',
  '1458/3908.jpeg',
  '1458/3909.jpeg',
  '1458/3910.jpeg',
  '1458/3911.jpeg',
  '1458/3912.jpeg',
  '1458/3913.jpeg',
  '1458/3914.jpeg',
  '1458/3915.jpeg',
  '1458/3916.jpeg',
  '1458/3917.jpeg',
  '1458/3918.jpeg',
  '1458/3919.jpeg',
  '1458/3920.jpeg',
  '1458/3921.jpeg',
  '1458/3922.jpeg',
  '1458/3923.jpeg',
  '1458/3924.jpeg',
  '1458/3925.jpeg',
  '1458/3926.jpeg',
  '1458/3927.jpeg',
  '1458/3928.jpeg',
  '1458/3929.jpeg',
  '1458/3930.jpeg',
  '1458/3931.jpeg',
  '1458/3932.jpeg',
  '1458/3933.jpeg',
  '1458/3934.jpeg',
  '1458/3935.jpeg',
  '1458/3936.jpeg',
  '1458/3937.jpeg',
  '1458/3938.jpeg',
  '1458/3939.jpeg',
  '1458/3940.jpeg',
  '1458/3941.jpeg',
  '1458/3942.jpeg',
  '1458/3943.jpeg',
  '1458/3944.jpeg',
  '1458/3945.jpeg',
  '1458/3946.jpeg',
  '1458/3947.jpeg',
  '1458/3948.jpeg',
  '1458/3949.jpe

- we have clip_id : list of the frame name e.g., '1458/3900.jpeg',  '1458/3901.jpeg',
- so we need to find the folder path for the clip and then convert it to video named by the clip_id 

In [5]:
big_dict = dict()
for data_id, framenames in obs_info_dict.items():
    big_dict[data_id] = framenames
for data_id, framenames in obs_info_dict_p.items():
    big_dict[data_id] = framenames
for data_id, framenames in obs_info_dict_pp.items():
    big_dict[data_id] = framenames


In [6]:
print(len(obs_info_dict))
print(len(obs_info_dict_p))
print(len(obs_info_dict_pp))
print(len(big_dict))

5655
608
607
6870


In [7]:
# this is for 84 size 
frame_folder = 'human_replay_screenshot_jpeg'
video_save_folder = "/home/sukai/Extrastorage/Montezuma_Project/video_clip_84"

In [7]:
# this is for 256 size 
frame_folder = '/home/sukai/Documents/Sukai_Project/MontezumaRevenge_human/atari_v1/screens/revenge'
video_save_folder = "/home/sukai/Extrastorage/Montezuma_Project/video_clip_5x_target_at_middle_256"

In [8]:
# this is for 84 size 
def create_video(video_id, framenames_arr, fps, size):
    output_path = os.path.join(video_save_folder,  f"{video_id}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (size, size))
    for framename in framenames_arr:
        img_path = os.path.join(frame_folder, framename)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        writer.write(img)
    writer.release()    
        

In [8]:
# this is for 256 size 
def create_video(video_id, framenames_arr, fps, size):
    output_path = os.path.join(video_save_folder,  f"{video_id}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (size, size))
    for framename in framenames_arr:
        framename = framename.split(".")[0] + ".png"
        img_path = os.path.join(frame_folder, framename)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (size, size))
        writer.write(img)
    writer.release()    
        

In [20]:
def get_next_name(pathname, number):
    # '1458/3906.jpeg'
    name, ext = pathname.split('.')
    foldername, filename = name.split('/')
    new_filename = str(int(filename) + number)
    newname = foldername+"/"+new_filename+"."+"png"
    return newname

# this is for 256 size 5x extend and 8 stride size version
def create_video(video_id, framenames_arr, fps, size):
    real_framenames_arr = []
    for i in range(0, len(framenames_arr), 8):
        real_framenames_arr.append(framenames_arr[i])
        
    real_frame_len = len(real_framenames_arr)
    # add before 
    frame_before_arr = []
    before_frame = real_framenames_arr[0]
    for i in range(real_frame_len * 2):
        before_frame = get_next_name(before_frame, -8)
        if os.path.exists(frame_folder + '/' + before_frame):
            frame_before_arr = [before_frame] + frame_before_arr 
        else:
            break
    
    # add after 
    frame_after_arr = []
    after_frame = real_framenames_arr[-1]
    for i in range(real_frame_len * 2):
        after_frame = get_next_name(after_frame, 8)
        if os.path.exists(frame_folder + '/'+ after_frame):
            frame_after_arr.append(after_frame)
        else:
            break
    
    starting_frame_ind = len(frame_before_arr)
    ending_frame_ind = len(frame_before_arr) + len(real_framenames_arr)
    real_framenames_arr =    frame_before_arr +  real_framenames_arr + frame_after_arr
        
    output_path = os.path.join(video_save_folder,  f"{video_id}_si_{starting_frame_ind}_ei_{ending_frame_ind}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (size, size))
    for framename in real_framenames_arr:
        framename = framename.split(".")[0] + ".png"
        img_path = os.path.join(frame_folder, framename)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (size, size))
        writer.write(img)
    writer.release()    

In [21]:
size = 256
video_save_folder_path = Path(video_save_folder)
video_save_folder_path.mkdir(parents=True, exist_ok=True)
for data_id, framenames in tqdm(big_dict.items()):
    create_video(data_id, framenames, 6.25, size)

  0%|          | 0/6870 [00:00<?, ?it/s]

In [ ]:
# somehow need to change codec for label studio to work
ffmpeg_folder_path = Path(f"/home/sukai/Extrastorage/Montezuma_Project/video_clip_ffmpeg_{size}")
ffmpeg_folder_path.mkdir(parents=True, exist_ok=True)
for data_id, framenames in tqdm(big_dict.items()):
    vid_name = os.path.join(video_save_folder, str(data_id)+".mp4")
    vid_name_output = os.path.join(f"/home/sukai/Extrastorage/Montezuma_Project/video_clip_ffmpeg_{size}", str(data_id)+".mp4")
    os.system(f"ffmpeg -i {vid_name} -vcodec libx264 {vid_name_output}")

# 2. generate video for atari human play dataset

In [14]:
import glob

In [15]:
frame_folder = '/home/sukai/Documents/Sukai_Project/MontezumaRevenge_human/atari_v1/screens/revenge'
frame_len = 150
video_save_folder = "/home/sukai/Extrastorage/Montezuma_Project/pretrain_visual_encoder_video_dataset_256"

In [16]:
Path(video_save_folder).mkdir(parents=True, exist_ok=True)

In [17]:
video_folder_path_arr = glob.glob(frame_folder+"/*/")

In [18]:
size = 256
fps=50
video_id = 0 
for video_folder_path in tqdm(video_folder_path_arr):
    start_image_ind = 0
    while True:
        output_path = os.path.join(video_save_folder,  f"{video_id}.mp4")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        

            
        img_arr = [] 
        writer_break_flag = False
        for ind in range(frame_len):
            # check if image exist 
            image_ind = start_image_ind + ind
            image_path = os.path.join(video_folder_path, f"{image_ind}.png")
            if os.path.exists(image_path):
                img = cv2.imread(image_path, cv2.IMREAD_COLOR)
                img = cv2.resize(img, (size, size))
                img_arr.append(img)
            else:
                writer_break_flag = True
                break
        if writer_break_flag:
            # means we do not write video 
            break 
        else:
            # we write video
            start_image_ind = image_ind + 1 # change start image ind 
            writer = cv2.VideoWriter(output_path, fourcc, fps, (size, size))
            for img in img_arr:
                writer.write(img)
            writer.release() 
            video_id += 1

  0%|          | 0/652 [00:00<?, ?it/s]